In [39]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding , SimpleRNN , Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pickle


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mohit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from nltk.corpus import stopwords

In [7]:
df=pd.read_excel("spam.xlsx")

## **Text Preprocessing**

In [8]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [10]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] , axis=1 , inplace=True)

In [11]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.columns=['target' , 'mail']

In [13]:
df.head()

,target,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
df.dropna(subset=['mail'] , inplace=True)

In [15]:
df['target']=df['target'].astype('str')
df['mail']=df['mail'].astype('str')

In [ ]:
ps=PorterStemmer()

In [17]:
review=re.sub('[^a-zA-Z]' , ' ' , df['mail'][0])
review=review.lower()
review=review.split()
review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
review=' '.join(review)
print(review)

go jurong point crazi avail bugi n great world la e buffet cine got amor wat


In [18]:
corpus=[]

for i in range(len(df)):
    review=re.sub('[^a-zA-Z]' , ' ' , df['mail'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
    

In [20]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [21]:
data=df.copy()
data.head()

,target,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [22]:
data['mail']=corpus

In [23]:
data.head()

,target,mail
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup final tkt st m...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though


In [24]:
label_encoder=LabelEncoder()
data['target']=label_encoder.fit_transform(data['target'])

## **0-ham , 1-spam**

In [25]:
data.shape

(5572, 2)

In [26]:
data.head()

,target,mail
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri wkli comp win fa cup final tkt st m...
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though


## **Settting embedding layer**

In [27]:
voc_size=10000
tokenizer=Tokenizer(num_words=voc_size , oov_token='<OOV>')
tokenizer

In [28]:
tokenizer.fit_on_texts(data['mail'])

In [29]:
tokenizer.word_index

{'<OOV>': 1,
 'u': 2,
 'call': 3,
 'go': 4,
 'get': 5,
 'ur': 6,
 'gt': 7,
 'lt': 8,
 'come': 9,
 'ok': 10,
 'day': 11,
 'free': 12,
 'know': 13,
 'love': 14,
 'like': 15,
 'time': 16,
 'want': 17,
 'good': 18,
 'got': 19,
 'text': 20,
 'send': 21,
 'txt': 22,
 'need': 23,
 'p': 24,
 'one': 25,
 'today': 26,
 'n': 27,
 'take': 28,
 'see': 29,
 'r': 30,
 'stop': 31,
 'home': 32,
 'think': 33,
 'repli': 34,
 'lor': 35,
 'k': 36,
 'sorri': 37,
 'tell': 38,
 'still': 39,
 'mobil': 40,
 'back': 41,
 'da': 42,
 'make': 43,
 'dont': 44,
 'phone': 45,
 'week': 46,
 'say': 47,
 'pleas': 48,
 'hi': 49,
 'new': 50,
 'work': 51,
 'pl': 52,
 'later': 53,
 'hope': 54,
 'ask': 55,
 'co': 56,
 'miss': 57,
 'msg': 58,
 'min': 59,
 'meet': 60,
 'c': 61,
 'messag': 62,
 'dear': 63,
 'night': 64,
 'wait': 65,
 'happi': 66,
 'well': 67,
 'thing': 68,
 'great': 69,
 'much': 70,
 'claim': 71,
 'oh': 72,
 'tri': 73,
 'give': 74,
 'wat': 75,
 'hey': 76,
 'number': 77,
 'b': 78,
 'friend': 79,
 'thank': 80,
 'y

In [30]:
sequences=tokenizer.texts_to_sequences(data['mail'])
sequences

[[4,
  3256,
  298,
  601,
  514,
  1070,
  27,
  69,
  249,
  818,
  84,
  2304,
  1071,
  19,
  3257,
  75],
 [10, 260, 542, 370, 2, 1543],
 [12,
  360,
  634,
  718,
  91,
  1544,
  896,
  299,
  1545,
  174,
  192,
  20,
  1544,
  211,
  360,
  267,
  764,
  22,
  234,
  61,
  295],
 [2, 175, 47, 300, 2305, 2, 61, 92, 47],
 [819, 33, 384, 765, 169, 151, 371],
 [635, 76, 1072, 46, 152, 41, 15, 301, 39, 1354, 10, 296, 764, 2306, 21, 1546],
 [90, 456, 15, 287, 495, 15, 2307, 3258],
 [162,
  766,
  1213,
  1213,
  1547,
  1835,
  1836,
  1837,
  385,
  820,
  515,
  666,
  767,
  79,
  820],
 [563, 516, 288, 143, 276, 2308, 85, 667, 71, 3, 71, 329, 2309, 372, 184],
 [40, 185, 2, 30, 975, 373, 250, 386, 40, 261, 12, 3, 40, 373, 56, 12],
 [165, 32, 153, 17, 154, 206, 976, 155, 36, 1073, 348, 26],
 [1548,
  244,
  91,
  95,
  349,
  22,
  2310,
  21,
  203,
  24,
  11,
  11,
  2311,
  295,
  34,
  496,
  476],
 [123,
  46,
  12,
  1549,
  85,
  2312,
  22,
  152,
  71,
  61,
  83,
  2313,

### **Padding**

In [31]:
padded_text=pad_sequences(sequences , maxlen=500 , padding='pre')
padded_text

array([[   0,    0,    0, ...,   19, 3257,   75],
       [   0,    0,    0, ...,  370,    2, 1543],
       [   0,    0,    0, ...,  234,   61,  295],
       ...,
       [   0,    0,    0, ..., 6240, 1067, 1380],
       [   0,    0,    0, ...,  830,  137,   12],
       [   0,    0,    0, ..., 2067,  356,  163]], dtype=int32)

In [32]:
padded_text.shape

(5572, 500)

In [33]:
data.head()

,target,mail
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri wkli comp win fa cup final tkt st m...
3,0,u dun say earli hor u c alreadi say
4,0,nah think goe usf live around though


In [34]:
X=padded_text
y=data['target']

In [35]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [36]:
print(X_train.shape , y_train.shape)
print(X_test.shape , y_test.shape)

(4457, 500) (4457,)
(1115, 500) (1115,)


In [37]:
print(X_train[0])
print(y_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [41]:
with open('tokenizer.pkl'  , 'wb') as file:
    pickle.dump(tokenizer , file)

## **Train Simple RNN**

In [66]:
dim=128
max_len=500

In [67]:
print(voc_size)
print(dim)
print(max_len)

10000
128
500


In [68]:
model=Sequential()
model.add(Embedding(input_dim=voc_size , output_dim=dim , input_length=max_len))
model.add(SimpleRNN(128 , activation='relu'))
model.add(Dense(1 , activation='sigmoid'))
model.build(input_shape=(None , max_len))
model.summary()

c:\Users\Mohit\Desktop\Deep Learning\RNN - Spam and Ham\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy']) 

In [71]:
log_dir = 'log/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [73]:
tensorflow_callback=TensorBoard(log_dir=log_dir , histogram_freq=1)

In [74]:
earlystopping=EarlyStopping(monitor='val_loss' , patience=5 , restore_best_weights=True)

In [76]:
model.fit(
    X_train , y_train , epochs=100 , batch_size=32,  
    validation_data=(X_test , y_test) ,
    callbacks=[tensorflow_callback , earlystopping]
)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 22s 103ms/step - accuracy: 0.8539 - loss: 0.4293 - val_accuracy: 0.9578 - val_loss: 0.1181
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - accuracy: 0.9839 - loss: 0.0553 - val_accuracy: 0.9883 - val_loss: 0.0425
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 17s 120ms/step - accuracy: 0.9969 - loss: 0.0129 - val_accuracy: 0.9821 - val_loss: 0.0638
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 176ms/step - accuracy: 0.9974 - loss: 0.0101 - val_accuracy: 0.9874 - val_loss: 0.0584
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 176ms/step - accuracy: 0.9983 - loss: 0.0076 - val_accuracy: 0.9883 - val_loss: 0.0531
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 17s 124ms/step - accuracy: 0.9999 - loss: 0.0014 - val_accuracy: 0.9892 - val_loss: 0.0444
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 15s 107ms/step - accuracy: 0.9999 - loss: 0.0016 - val_accuracy: 0.9883 - val_loss: 0.0551


In [77]:
model.save('model.h5')

## **Model Details**

In [78]:
model.layers

[<Embedding name=embedding, built=True>,
 <SimpleRNN name=simple_rnn, built=True>,
 <Dense name=dense, built=True>]

In [79]:
model.name

'sequential_1'

In [42]:
X_train[0].shape

(500,)

## **Checking Tensorboard**

In [4]:
%load_ext tensorboard

In [6]:
%tensorboard --logdir log/fit20250629-161831

Reusing TensorBoard on port 6006 (pid 21928), started 0:15:02 ago. (Use '!kill 21928' to kill it.)